In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import time
import matplotlib.pyplot as plt
import math
import copy
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from tensorflow.keras.layers import Input
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout

# Índice

1. Coleta de dados
   1. Escolha das Criptomoedas que serão trabalhadas
   2. Criação do dicionário 'data'
   3. Criação do dicionário 'data_complete'
2. Tratamento das tabelas
   1. Criar vetor de dias - First Transform
   2. Separar em treinamento e teste - Permitir Bagging
   3. Bagging
3. Machine Learning
   1. Função do Modelo
   2. Treinamento com Paralelismo e Geração dos Expected Values
4. Preparação para Portifólio
   1. Preparar data_complete para conter volatilidade
   2. Criar vetor de médias
   3. Criar matriz de covariâncias
5. Alocação de Portifólio
   1. Para cada intervalo de tempo, selecionar pesos de portifólio
   2. Calcular retorno desse portifólio no intervalo de tempo
   3. Salvar retorno e analisar resultados

# 1. Coleta de dados

## 1.1 Escolha das Criptomoedas que serão trabalhadas

In [2]:
# Criptomoedas que vamos escolher para nosso portifólio
cryptos = [
    'BTC-USD', 'ETH-USD','LTC-USD', 'ADA-USD',
    'DOT-USD', 'LINK-USD','SOL-USD',
    'TRX-USD'
]

## 1.2 Criação do dicionário 'data'

In [3]:
## Função para calcular o retorno logarítmico
def log_return(series):
    return np.log(series / series.shift(1))

In [4]:
data = {}
for crypto in cryptos:
    # Baixar dados e calcular retorno logarítmico
    df = yf.download(crypto, start='2020-08-21', end=pd.to_datetime("today").strftime("%Y-%m-%d"))
    df['Return'] = log_return(df['Close']).shift(-2)

    # Construir DataFrame final, pegando o retorno do dia anterior e o atual
    df_final = pd.DataFrame({
        'Crypto_Return_Day_1': df['Return'].shift(1),
        'Crypto_Return_Today': df['Return']
    }).dropna()

    data[crypto] = df_final[['Crypto_Return_Today']]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


## 1.3 Criação do dicionário 'data_complete'

In [5]:
data_complete = {}
for crypto in cryptos:
    # Baixar dados e calcular retorno logarítmico
    df = yf.download(crypto, start='2020-08-21', end=pd.to_datetime("today").strftime("%Y-%m-%d"))
    df['Return'] = log_return(df['Close']).shift(-2)

    # Construir DataFrame final, pegando o retorno do dia anterior e o atual
    df_final = pd.DataFrame({
        'Crypto_Return_Day_1': df['Return'].shift(1),
        'Crypto_Return_Today': df['Return']
    }).dropna()

    # Adicionar a coluna 'exp_value' com valores NaN
    df_final['exp_value'] = np.nan

    # Armazenar no dicionário
    data_complete[crypto] = df_final[['Crypto_Return_Today', 'exp_value']]

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [6]:
data['ETH-USD']

,Crypto_Return_Today
Date,
2020-08-22,0.041930
2020-08-23,-0.060975
2020-08-24,0.006399
2020-08-25,-0.009969
2020-08-26,0.034022
...,...
2024-10-11,-0.003578
2024-10-12,0.063288
2024-10-13,-0.008741


# 2. Tratamento das tabelas

## 2.1 Criar vetor de dias - First Transform

# > Aqui eu preciso dizer: Essa etapa first transform é inutil e está apenas consumindo tempo.
# > Solução: Sumir com o FirstTransform, ou atualizar ele.
# > Para cada dia. adicionar o treinamento daquele dia no modelo, e então calcular o exp_value para o próximo

In [7]:
W = 290      ####     HIPER PARAMETRO     #####
# Criando a função First Transformation, que recebe uma tabela contendo os retornos de todos os dias dos ultimos X anos e retorno um vetor, que cada elemento
# é uma tabela do retorno de W+1 dias.
def FirstTransform(df, W):
    vet = []
    Linhas, Colunas = df.shape
    for i in range(Linhas, W+1, -1):
        vet.append(df.iloc[(i-W-1):i])
    return vet

In [8]:
for crypto in cryptos:
    data[crypto] = FirstTransform(data[crypto])

In [9]:
data['ETH-USD'][0]

,Crypto_Return_Today
Date,
2020-08-22,0.041930
2020-08-23,-0.060975
2020-08-24,0.006399
2020-08-25,-0.009969
2020-08-26,0.034022
...,...
2024-10-11,-0.003578
2024-10-12,0.063288
2024-10-13,-0.008741


## 2.2 Separar em treinamento e teste - Permitir Bagging

In [10]:
test_target = {}
for crypto in cryptos:
    row = []
    for i in range(len(data[crypto])):
        row.append((pd.DataFrame(data[crypto][i].iloc[-1])).T)
        data[crypto][i] = data[crypto][i].drop(data[crypto][i].index[-1])
    test_target[crypto] = row

In [11]:
data['ETH-USD'][0], test_target['ETH-USD'][0]

(            Crypto_Return_Today
 Date                           
 2020-08-22             0.041930
 2020-08-23            -0.060975
 2020-08-24             0.006399
 2020-08-25            -0.009969
 2020-08-26             0.034022
 ...                         ...
 2024-10-10             0.016288
 2024-10-11            -0.003578
 2024-10-12             0.063288
 2024-10-13            -0.008741
 2024-10-14             0.001947
 
 [1515 rows x 1 columns],
             Crypto_Return_Today
 2024-10-15            -0.002619)

## 2.3 Bagging

In [12]:
# Definindo uma função bagging, que recebe um dataframe e retorna um vetor de dataframes.
def Bagging(df, n, gamma):
    df_bagged = []
    for i in range(gamma):
        aux = df.sample(n = n, random_state = i)
        aux = aux.sort_index()
        df_bagged.append(aux)
    return df_bagged

# 3. Machine Learning

* vamos peimeiro fazer do modo 1 para todos os dias
* depois aplicar para os ultimos 290 dias o modo 2 (contém bagging e purge K-Fold-CV)

## 3.1 Função do Modelo


In [13]:
# Função para criar o modelo
def create_model(a, b):
    model = Sequential()
    model.add(Input(shape=(a, b)))  # Define a camada de entrada
    model.add(LSTM(units=50, return_sequences=True))  # Primeira camada LSTM
    model.add(Dropout(0.2))  # Dropout para prevenir overfitting
    model.add(LSTM(units=50, return_sequences=False))  # Segunda camada LSTM
    model.add(Dropout(0.2))
    model.add(Dense(units=25))  # Densa intermediária
    model.add(Dense(units=1))  # Camada de saída
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Função do modelo LSTM
def Model(df, model=None):
    # Normalizando o dataframe
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)
    df_array = np.array(df_scaled)

    # Transformando no formato LSTM
    X_lstm = []
    y_lstm = []
    timesteps = 60
    for i in range(timesteps, len(df_array)):
        X_lstm.append(df_array[i-timesteps:i, :])  # Pega 'timesteps' linhas anteriores
        y_lstm.append(df_scaled[i])  # Alvo é o valor do dia seguinte
    X_lstm = np.array(X_lstm)
    y_lstm = np.array(y_lstm)

    # Separar o X_train (todos exceto o último elemento)
    X_train = X_lstm[:-1]
    y_train = y_lstm[:-1]

    # X_test e y_test
    X_test = X_lstm[-1:]
    y_test = y_lstm[-1:]

    # Treinar o modelo específico para a criptomoeda
    early_stopping = EarlyStopping(monitor='loss', patience=3)  # Para o treinamento quando a perda parar de melhorar
    model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stopping])

    # Fazer a previsão
    y_hat_scaled = model.predict(X_test)
    y_hat = scaler.inverse_transform(y_hat_scaled)[0][0]

    return y_hat

##  3.2 Treinamento com Paralelismo e Geração dos Expected Values

In [14]:
def Add(y_hat, data_complete, ind, crypto):
    data_complete[crypto].at[ind, 'exp_value'] = y_hat

In [15]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print("Usando GPU:", physical_devices[0])
else:
    print("Usando CPU")

Usando CPU


In [16]:
len(data['ETH-USD'])

1456

: 

In [ ]:
# Função que processa cada criptomoeda (paralelismo será aplicado aqui)
def process_crypto(crypto, data, test_target, data_complete, D):
    # Criar um modelo para cada criptomoeda
    model = create_model(60, 1)  # 15 time steps, 1 feature (ajuste conforme necessário)

    for day in range(len(data[crypto]) - 60 , len(data[crypto]) - 60 - D, -1):
        df_train = data[crypto][day].copy()  # df é uma tabela de 300 linhas e uma coluna
        df_test = test_target[crypto][day].copy()  # df_test é uma tabela com 1 linha e uma coluna
        df = pd.concat([df_train, df_test], ignore_index=True)  # Unindo as duas tabelas

        y_hat = Model(df, model=model)  # Chamar o modelo para previsão
        Add(y_hat, data_complete, df_test.index[0], crypto)  # Adicionar o resultado na tabela final
# Número de dias que vamos testar o modelo
D = 290
# Criar o executor para rodar as criptomoedas em paralelo
with ThreadPoolExecutor() as executor:
    # Executar o processo para cada criptomoeda simultaneamente
    futures = [
        executor.submit(process_crypto, crypto, data, test_target, data_complete, D)
        for crypto in cryptos
    ]

    # Aguardar a conclusão de todas as threads
    for future in futures:
        future.result()

print("Processamento em paralelo finalizado.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 933ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 690ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 711ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 758ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 

In [23]:
data_complete['ETH-USD']

,Crypto_Return_Today,exp_value
Date,,
2020-08-22,0.041930,NaN
2020-08-23,-0.060975,NaN
2020-08-24,0.006399,NaN
2020-08-25,-0.009969,NaN
2020-08-26,0.034022,NaN
...,...,...
2024-10-11,-0.003578,0.006798
2024-10-12,0.063288,-0.000902
2024-10-13,-0.008741,0.010335


In [48]:
for crypto in cryptos:
    # Calcular a diferença entre Crypto_Return_Today e exp_value
    # Calcular o desvio padrão do Crypto_Return_Today nos últimos 30 dias
    data_complete[crypto]['volatility'] = data_complete[crypto]['Crypto_Return_Today'].rolling(window=30).std()

for crypto in cryptos:
    # Define o nome do arquivo como o nome da criptomoeda seguido de .csv
    filename = f"{crypto}.csv"
    
    # Salva o DataFrame data_complete[crypto] no arquivo .csv
    data_complete[crypto].to_csv(filename, index=False)

    print(f"Arquivo {filename} salvo com sucesso!")

## 3.3 Método 2 para os últimos 200 dias

* Aplicar bagging
* Aplicar CV e Otimização de HP

In [ ]:
# for day in data[crypto]:
#       day = bagging(day)

# for day in data[crypto]:
#       for sample in day:
#               sample = CV(sample) # transforma sample em um vetor de folds. Ultimos 10% do fold são validation e 90% são pure train

# for day in data[crypto]:
#       for sample in day:
#               for fold in sample:
#                       y_hat = model(fold)
#               y_hat_mean = y_hat / len(sample)
#       y_hat_mean_mean = y_hat_mean / len(day)